In [ ]:
!pip install cohere

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the CSV file into a DataFrame
df = pd.read_csv('Your_data.csv')

# Split the dataset into 80% train and 20% test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Write the train and test DataFrames to separate CSV files
train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)


In [ ]:
import csv
import json

# Function to read CSV file and convert data to JSON Lines format
def csv_to_jsonl(csv_file, output_file):
    # Open JSON Lines output file
    with open(output_file, 'w') as jsonl_file:
        # Read CSV file
        with open(csv_file, 'r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                # Create JSON object for system message
                system_message = {"role": "System", "content": "You are an expert in determining biological age of people based on their PPG features."}
                # Create JSON object for user message
                user_message = {"role": "User", "content": row["text"]}
                # Create JSON object for chatbot message with age
                chatbot_message = {"role": "Chatbot", "content": row["Age"]}
                # Construct message set
                message_set = {"messages": [system_message, user_message, chatbot_message]}
                # Write message set to JSON Lines file
                jsonl_file.write(json.dumps(message_set, indent=2) + '\n')

# Example usage:
csv_file = "put your test and train data"  # Replace with your CSV file path
jsonl_output_file = "output.jsonl"  # Replace with desired output file path
csv_to_jsonl(csv_file, jsonl_output_file)


In [ ]:
import csv
import cohere
import time

# Initialize cohere client
co = cohere.Client('Your API Key')

# Function to send message and get response
def get_response(message):
    response = co.chat(
        model='Your Pre-trained Model ID',
        message=message)
    return response.text

# Read input CSV file
input_file = 'Your input.csv'

# Output CSV file
output_file = 'predict.csv'

# Counter to keep track of API calls
api_call_counter = 0

with open(input_file, 'r') as csv_input, open(output_file, 'a', newline='') as csv_output:
    csv_reader = csv.reader(csv_input)
    csv_writer = csv.writer(csv_output)

    # If the output CSV file is empty, write headers
    if csv_output.tell() == 0:
        csv_writer.writerow(['Input', 'Response'])

    for row in csv_reader:
        input_message = row[0]  # Assuming the message is in the first column of each row

        # Get response for the input message
        response_text = get_response(input_message)

        # Write input message and response to output CSV
        csv_writer.writerow([input_message, response_text])

        # Increment API call counter
        api_call_counter += 1

        # Check if 20 API calls have been made
        if api_call_counter % 20 == 0:
            # Introduce a 60-second delay
            time.sleep(60)
        else:
            # Introduce a 1-second delay for all other API calls
            time.sleep(1)

print("CSV file with responses generated successfully.")